# Global settings

In [1]:
device = 'cuda'
bs = 64

# Data proc

In [27]:
import pandas as pd
import numpy as np
import torch
import itertools
from make_matrix import *
from torch.utils.data import TensorDataset, DataLoader, Sampler
import torch.nn as nn
import torch.nn.functional as F

In [3]:
df = pd.read_csv('flu_train.csv')

In [4]:
df.head()

,Unnamed: 0,virusUprot,humanUprot,edge
0,0,P03433,P49736,1.0
1,1,P03433,P15311,0.0
2,2,P03433,P11142,0.0
3,3,P03433,Q86U42,0.0
4,4,P03433,P33992,1.0


In [5]:
vs = df['virusUprot'].unique()
hs = df['humanUprot'].unique()

In [6]:
df.head()

,Unnamed: 0,virusUprot,humanUprot,edge
0,0,P03433,P49736,1.0
1,1,P03433,P15311,0.0
2,2,P03433,P11142,0.0
3,3,P03433,Q86U42,0.0
4,4,P03433,P33992,1.0


In [7]:
vs.sort()
hs.sort()
vs_map = {v:i for i,v in enumerate(vs)}
hs_map = {h:i for i, h in enumerate(hs)}

In [8]:
df.virusUprot = df.virusUprot.apply(lambda x : vs_map[x])
df.humanUprot = df.humanUprot.apply(lambda x : hs_map[x])
df.head()

,Unnamed: 0,virusUprot,humanUprot,edge
0,0,52,946,1.0
1,1,52,634,0.0
2,2,52,577,0.0
3,3,52,1785,0.0
4,4,52,807,1.0


In [9]:
len(df.loc[df['edge'] == 1]) / len(df)

0.018504970365673678

### Make a dataloader

In [10]:
X = list(zip(df.virusUprot.values, df.humanUprot.values))

In [11]:
y = df['edge'].values

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [14]:
y_train[y_train == 1]

array([1., 1., 1., ..., 1., 1., 1.])

In [15]:
def arr_to_torch(arr, dtype):
    t =  torch.from_numpy(np.array(arr)).type(dtype).to(device)
    return t

In [44]:
class StratifiedSampler(Sampler):
    """Stratified Sampling
    Provides equal representation of target classes in each batch
    """
    def __init__(self, class_vector, batch_size):
        """
        Arguments
        ---------
        class_vector : torch tensor
            a vector of class labels
        batch_size : integer
            batch_size
        """
        self.n_splits = int(class_vector.size(0) / batch_size)
        self.class_vector = class_vector

    def gen_sample_array(self):
        try:
            from sklearn.model_selection import StratifiedShuffleSplit
        except:
            print('Need scikit-learn for this functionality')
        import numpy as np
        
        s = StratifiedShuffleSplit(n_splits=self.n_splits, test_size=0.5)
        X = torch.randn(self.class_vector.size(0),2).numpy()
        y = self.class_vector.numpy()
        s.get_n_splits(X, y)

        train_index, test_index = next(s.split(X, y))
        return np.hstack([train_index, test_index])

    def __iter__(self):
        return iter(self.gen_sample_array())

    def __len__(self):
        return len(self.class_vector)

In [45]:
sampler = StratifiedSampler(torch.Tensor([1.0, 0.0]), bs)

In [56]:
train_dset = TensorDataset(arr_to_torch(X_train, torch.long), arr_to_torch(y_train, torch.float32))
# train_loader = DataLoader(train_dset, batch_size=bs, sampler=sampler)
train_loader = DataLoader(train_dset, batch_size=bs, shuffle=True)

In [57]:
val_dset = TensorDataset(arr_to_torch(X_val, torch.long), arr_to_torch(y_val, torch.float32))
# val_loader = DataLoader(val_dset, batch_size=bs, sampler=sampler)
val_loader = DataLoader(val_dset, batch_size=bs, shuffle=True)

In [58]:
test_dset = TensorDataset(arr_to_torch(X_test, torch.long), arr_to_torch(y_test, torch.float32))
# test_loader = DataLoader(test_dset, batch_size=bs, sampler=sampler)
test_loader = DataLoader(test_dset, batch_size=bs, shuffle=True)

# Model

In [59]:
def l2_regularize(array):
    loss = torch.sum(array ** 2.0)
    return loss

In [60]:
class MF(nn.Module):
    def __init__(self, n_virus, n_human, k=18, c_vector=1.0, c_bias=1.0, writer=None):
        super(MF, self).__init__()
        self.writer = writer
        self.k = k
        self.n_virus = n_virus
        self.n_human = n_human
        self.c_bias = c_bias
        self.c_vector = c_vector
        
        self.virus = nn.Embedding(n_virus, k)
        self.human = nn.Embedding(n_human, k)
        
        # We've added new terms here:
        self.bias_virus = nn.Embedding(n_virus, 1)
        self.bias_human = nn.Embedding(n_human, 1)
        self.bias = nn.Parameter(torch.ones(1))
        
#         self.affine = nn.Linear(in_features=self.k, out_features=1)
#         self.logistic = nn.Sigmoid()
    
    def forward(self, train_x):
        virus_id = train_x[:, 0]
        human_id = train_x[:, 1]
        vector_virus = self.virus(virus_id)
        vector_human = self.human(human_id)
        
        # Pull out biases
        bias_virus = self.bias_virus(virus_id).squeeze()
        bias_human = self.bias_human(human_id).squeeze()
        biases = (self.bias + bias_virus + bias_human)
        
        ui_interaction = torch.sum(vector_virus * vector_human, dim=1)
        
        prediction = ui_interaction + biases
        return prediction
    
    def loss(self, prediction, target):
#         loss_mse = F.binary_cross_entropy_with_logits(prediction, target.squeeze())
        loss_mse = F.mse_loss(prediction, target.squeeze())
    
        # Add new regularization to the biases
        prior_bias_virus =  l2_regularize(self.bias_virus.weight) * self.c_bias
        prior_bias_human = l2_regularize(self.bias_human.weight) * self.c_bias
        
        prior_virus =  l2_regularize(self.virus.weight.data) * self.c_vector
        prior_human = l2_regularize(self.human.weight.data) * self.c_vector
        total = loss_mse + prior_virus + prior_human + prior_bias_virus + prior_bias_human
        return total

# Trainer

In [61]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss, Accuracy, Precision, Recall
from tensorboardX import SummaryWriter
from ignite.metrics import MeanSquaredError, Loss
from ignite.contrib.metrics import AveragePrecision, ROC_AUC

from datetime import datetime

In [62]:
# Hyperparameters
lr = 1e-3
k =20
# New parameter for regularizing bias
c_bias = 1e-4
c_vector = 1e-4
batchsize = bs
log_dir = 'runs/simple_mf_02_bias_' + str(datetime.now()).replace(' ', '_')
print(log_dir)

runs/simple_mf_02_bias_2019-05-18_21:55:58.984736


In [63]:
writer = SummaryWriter(log_dir=log_dir)
# crit = nn.MSELoss()

model = MF(len(vs_map), len(hs_map), writer=writer, k=k, c_bias=c_bias, c_vector=c_vector)
crit = model.loss
model.cuda()
optimizer = torch.optim.Adam(model.parameters())
trainer = create_supervised_trainer(model, optimizer, model.loss)
metrics = {'loss': Loss(crit), 'ap': AveragePrecision(), "acc": Accuracy(), "roc": ROC_AUC()}
evaluator = create_supervised_evaluator(model, metrics=metrics)

def log_training_loss(engine, log_interval=400):
    epoch = engine.state.epoch
    itr = engine.state.iteration
    fmt = "Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
    msg = fmt.format(epoch, itr, len(train_loader), engine.state.output)
    model.itr = itr
    if itr % log_interval == 0:
        print(msg)
#         metrics = trainer.state.metrics
#         mse = metrics['loss']
#         avg_precision = metrics['ap']
#         accuracy = metrics['acc']
#         roc = metrics['roc']
#         writer.add_scalar("training/mse", mse, engine.state.epoch)
#         writer.add_scalar("training/ap", avg_precision, engine.state.epoch)
#         writer.add_scalar("training/accuracy", accuracy, engine.state.epoch)
#         writer.add_scalar("training/roc", roc, engine.state.epoch)

trainer.add_event_handler(event_name=Events.ITERATION_COMPLETED, handler=log_training_loss)

def log_validation_results(engine):
    evaluator.run(val_loader)
    metrics = evaluator.state.metrics
    mse = metrics['loss']
    avg_precision = metrics['ap']
    accuracy = metrics['acc']
    roc = metrics['roc']
    print("Epoch[{}] Validation MSE: {:.2f} Avg Prec: {:.2f} acc: {:.2f} ROC: {:.2f} "
          .format(engine.state.epoch, mse, avg_precision, accuracy, roc))
    writer.add_scalar("validation/mse", mse, engine.state.epoch)
    writer.add_scalar("validation/avg_precision", avg_precision, engine.state.epoch)
    writer.add_scalar("validation/accuracy", accuracy, engine.state.epoch)
    writer.add_scalar("validation/roc", roc, engine.state.epoch)

    
def log_test_results(engine):
    evaluator.run(test_loader)
    metrics = evaluator.state.metrics
    mse = metrics['loss']
    avg_precision = metrics['ap']
    accuracy = metrics['acc']
    roc = metrics['roc']

    print("TEST: Epoch[{}] Validation MSE: {:.2f} Avg Prec: {:.2f} acc: {:.2f} ROC: {:.2f}"
          .format(engine.state.epoch, mse, avg_precision, accuracy, roc))


trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=log_validation_results)
trainer.add_event_handler(event_name=Events.COMPLETED, handler=log_test_results)


model

MF(
  (virus): Embedding(203, 20)
  (human): Embedding(2727, 20)
  (bias_virus): Embedding(203, 1)
  (bias_human): Embedding(2727, 1)
)

In [64]:
model.to(device)

MF(
  (virus): Embedding(203, 20)
  (human): Embedding(2727, 20)
  (bias_virus): Embedding(203, 1)
  (bias_human): Embedding(2727, 1)
)

In [65]:
trainer.run(train_loader, max_epochs=500)

Epoch[1] Iteration[400/6228] Loss: 28.55
Epoch[1] Iteration[800/6228] Loss: 23.57
Epoch[1] Iteration[1200/6228] Loss: 23.04
Epoch[1] Iteration[1600/6228] Loss: 21.61
Epoch[1] Iteration[2000/6228] Loss: 19.75
Epoch[1] Iteration[2400/6228] Loss: 18.35
Epoch[1] Iteration[2800/6228] Loss: 16.62
Epoch[1] Iteration[3200/6228] Loss: 15.01
Epoch[1] Iteration[3600/6228] Loss: 12.30
Epoch[1] Iteration[4000/6228] Loss: 10.92
Epoch[1] Iteration[4400/6228] Loss: 12.47
Epoch[1] Iteration[4800/6228] Loss: 11.65
Epoch[1] Iteration[5200/6228] Loss: 10.62
Epoch[1] Iteration[5600/6228] Loss: 11.34
Epoch[1] Iteration[6000/6228] Loss: 8.51


/home/roncal/.conda/envs/deeplearning/lib/python3.6/site-packages/sklearn/metrics/ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/roncal/.conda/envs/deeplearning/lib/python3.6/site-packages/ignite/metrics/epoch_metric.py:76: RuntimeWarning: Probably, there can be a problem with `compute_fn`:
 Only one class present in y_true. ROC AUC score is not defined in that case.
  RuntimeWarning)


Epoch[1] Validation MSE: 8.06 Avg Prec: 0.02 acc: 0.22 ROC: 0.51 
Epoch[2] Iteration[6400/6228] Loss: 7.38
Epoch[2] Iteration[6800/6228] Loss: 6.85
Epoch[2] Iteration[7200/6228] Loss: 6.38
Epoch[2] Iteration[7600/6228] Loss: 5.94
Epoch[2] Iteration[8000/6228] Loss: 6.28
Epoch[2] Iteration[8400/6228] Loss: 5.87
Epoch[2] Iteration[8800/6228] Loss: 5.25
Epoch[2] Iteration[9200/6228] Loss: 4.79
Epoch[2] Iteration[9600/6228] Loss: 4.60
Epoch[2] Iteration[10000/6228] Loss: 4.35
Epoch[2] Iteration[10400/6228] Loss: 4.48
Epoch[2] Iteration[10800/6228] Loss: 4.20
Epoch[2] Iteration[11200/6228] Loss: 3.85
Epoch[2] Iteration[11600/6228] Loss: 4.01
Epoch[2] Iteration[12000/6228] Loss: 3.88
Epoch[2] Iteration[12400/6228] Loss: 3.69
Epoch[2] Validation MSE: 3.72 Avg Prec: 0.03 acc: 0.78 ROC: 0.60 
Epoch[3] Iteration[12800/6228] Loss: 3.64
Epoch[3] Iteration[13200/6228] Loss: 3.56
Epoch[3] Iteration[13600/6228] Loss: 3.55
Epoch[3] Iteration[14000/6228] Loss: 3.56
Epoch[3] Iteration[14400/6228] Loss: 

/home/roncal/.conda/envs/deeplearning/lib/python3.6/site-packages/sklearn/metrics/ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/roncal/.conda/envs/deeplearning/lib/python3.6/site-packages/ignite/metrics/epoch_metric.py:76: RuntimeWarning: Probably, there can be a problem with `compute_fn`:
 Only one class present in y_true. ROC AUC score is not defined in that case.
  RuntimeWarning)


Epoch[6] Validation MSE: 2.86 Avg Prec: 0.31 acc: 0.98 ROC: 0.85 
Epoch[7] Iteration[37600/6228] Loss: 2.85
Epoch[7] Iteration[38000/6228] Loss: 2.84
Epoch[7] Iteration[38400/6228] Loss: 2.85
Epoch[7] Iteration[38800/6228] Loss: 2.83
Epoch[7] Iteration[39200/6228] Loss: 2.81
Epoch[7] Iteration[39600/6228] Loss: 2.81
Epoch[7] Iteration[40000/6228] Loss: 2.79
Epoch[7] Iteration[40400/6228] Loss: 2.80
Epoch[7] Iteration[40800/6228] Loss: 2.79
Epoch[7] Iteration[41200/6228] Loss: 2.78
Epoch[7] Iteration[41600/6228] Loss: 2.76
Epoch[7] Iteration[42000/6228] Loss: 2.75
Epoch[7] Iteration[42400/6228] Loss: 2.76
Epoch[7] Iteration[42800/6228] Loss: 2.76
Epoch[7] Iteration[43200/6228] Loss: 2.75
Epoch[7] Validation MSE: 2.74 Avg Prec: 0.36 acc: 0.98 ROC: 0.86 
Epoch[8] Iteration[43600/6228] Loss: 2.76
Epoch[8] Iteration[44000/6228] Loss: 2.72
Epoch[8] Iteration[44400/6228] Loss: 2.73
Epoch[8] Iteration[44800/6228] Loss: 2.72
Epoch[8] Iteration[45200/6228] Loss: 2.71
Epoch[8] Iteration[45600/622

...

/home/roncal/.conda/envs/deeplearning/lib/python3.6/site-packages/sklearn/metrics/ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/roncal/.conda/envs/deeplearning/lib/python3.6/site-packages/ignite/metrics/epoch_metric.py:76: RuntimeWarning: Probably, there can be a problem with `compute_fn`:
 Only one class present in y_true. ROC AUC score is not defined in that case.
  RuntimeWarning)


Epoch[33] Validation MSE: 1.45 Avg Prec: 0.43 acc: 0.98 ROC: 0.85 
Epoch[34] Iteration[205600/6228] Loss: 1.44
Epoch[34] Iteration[206000/6228] Loss: 1.44
Epoch[34] Iteration[206400/6228] Loss: 1.44
Epoch[34] Iteration[206800/6228] Loss: 1.43
Epoch[34] Iteration[207200/6228] Loss: 1.43
Epoch[34] Iteration[207600/6228] Loss: 1.43
Epoch[34] Iteration[208000/6228] Loss: 1.42
Epoch[34] Iteration[208400/6228] Loss: 1.42
Epoch[34] Iteration[208800/6228] Loss: 1.42
Epoch[34] Iteration[209200/6228] Loss: 1.42
Epoch[34] Iteration[209600/6228] Loss: 1.42
Epoch[34] Iteration[210000/6228] Loss: 1.41
Epoch[34] Iteration[210400/6228] Loss: 1.42
Epoch[34] Iteration[210800/6228] Loss: 1.41
Epoch[34] Iteration[211200/6228] Loss: 1.40
Epoch[34] Iteration[211600/6228] Loss: 1.40
Epoch[34] Validation MSE: 1.42 Avg Prec: 0.43 acc: 0.98 ROC: 0.85 
Epoch[35] Iteration[212000/6228] Loss: 1.41
Epoch[35] Iteration[212400/6228] Loss: 1.40
Epoch[35] Iteration[212800/6228] Loss: 1.40
Epoch[35] Iteration[213200/622

/home/roncal/.conda/envs/deeplearning/lib/python3.6/site-packages/sklearn/metrics/ranking.py:526: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/roncal/.conda/envs/deeplearning/lib/python3.6/site-packages/ignite/metrics/epoch_metric.py:76: RuntimeWarning: Probably, there can be a problem with `compute_fn`:
 Only one class present in y_true. ROC AUC score is not defined in that case.
  RuntimeWarning)


Epoch[35] Validation MSE: 1.39 Avg Prec: 0.42 acc: 0.98 ROC: 0.85 
Epoch[36] Iteration[218000/6228] Loss: 1.37
Epoch[36] Iteration[218400/6228] Loss: 1.37
Epoch[36] Iteration[218800/6228] Loss: 1.36
Epoch[36] Iteration[219200/6228] Loss: 1.37
Epoch[36] Iteration[219600/6228] Loss: 1.36
Epoch[36] Iteration[220000/6228] Loss: 1.36
Epoch[36] Iteration[220400/6228] Loss: 1.35
Epoch[36] Iteration[220800/6228] Loss: 1.35
Epoch[36] Iteration[221200/6228] Loss: 1.35
Epoch[36] Iteration[221600/6228] Loss: 1.35


KeyboardInterrupt: 